<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Finetune and convert Llama3 to an Ollama Modelfile 🤙

Hi everyone!

In the last couple months, Ollama has taken the AI community by a storm. It's one of the easiest ways of interacting with open souce models and has a ton of support from the community. One thing we havn't seen online is end-to-end guides on how to finetune a model and then convert it to an Ollama model so it can be used anywhere...so we decided to build one :)

In this notebook, we will do the following. 
1. Finetune Llama3-8B instruct using a novel finetuning method called ORPO. This combines Supvervised-Finetuning with Direct Preference Optimization and happens all in 1 step
2. We will install `llama.cpp` and convert our model to `gguf` format and talk about the different options (and how LoRA plays in here)
3. We will write our Ollama `Modelfile` which can be thought of as a Dockerfile for Ollama models
4. We will locally test and then push our model to the Ollama hub
5. We will pull our model on a separate machine and launch it for inference

Grab some snacks, buckle-up, and let it rip 🤙!

#### Help us make this tutorial better! Please provide feedback on the [Discord channel](https://discord.gg/T9bUNqMS8d) or on [X](https://x.com/brevdev).

A note about running Jupyter Notebooks: Press Shift + Enter to run a cell. A * in the left-hand cell box means the cell is running. A number means it has completed. If your Notebook is acting weird, you can interrupt a too-long process by interrupting the kernel (Kernel tab -> Interrupt Kernel) or even restarting the kernel (Kernel tab -> Restart Kernel). Note restarting the kernel will require you to run everything from the beginning.

# Phase 1: Finetune Llama3 with ORPO

So far, we've released guides on finetuning Llama3 with 2 different approaches
1. [Supervised Finetuning](https://github.com/brevdev/notebooks/blob/main/llama3_finetune_inference.ipynb)
2. [Direct Preference Optimization](https://github.com/brevdev/notebooks/blob/main/llama3dpo.ipynb)

If you havn't taken a look at these, I suggest skimming through the code for the SFT notebook and the explanation at the top of the DPO notebook. TLDR: Most of the finetuning in guides and online use SFT. SFT is good at adapting a model to domain-specific ouput but might also increase the probability of generating outputs that are not as desirable. To solve this issue, we can then perform a process known as preference alignment. This can be done using RLHF or DPO. However, these are both computationally expensive. 

Enter Odds Ratio Policy Optimization. At a high level, this combines SFT and DPO into 1 neat step which weakly penalizes rejected responses while strongly rewarding preferred responses. Here we optimize for two objectives at once: learning domain-specific output AND aligning the output with out preferences. If you'd like to dive deeper into ORPO, check out MLabonne's [fantastic guide](https://huggingface.co/blog/mlabonne/orpo-llama-3) on HuggingFace

![ORPO](https://i.imgur.com/ftrth4Q.png)


## Install Dependancies

In [ ]:
!pip install bitsandbytes
!pip install wandb
!pip install transformers
!pip install peft
!pip install accelerate
!pip install trl

In [ ]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer

## Load Model, Tokenizer, and Dataaset

We'll be working with the Llama3-8B instruct model. But the steps are almost the same with any other model if you decide to finetune with a LoRA adapter. 

In [6]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "brevity-adapter"

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)

In [ ]:
dataset_name = "mlabonne/orpo-dpo-mix-40k"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(150))

def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)

Take a look at the dataset by uncommenting the code block below

In [ ]:
# for k in dataset['train'].features.keys():
#     print(k)
#     print("---------")
#     print(dataset['train'][1][k])
#     print('\n')

## Set up ORPO Training

The ORPO trainer looks very similar to the SFTTrainer and the DPO trainer. We set our config parameters and start our training run. Notice that the run is very short. In order to increase it, you can increase the `num_train_epochs` or add a `max_steps` argument

In [ ]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    max_steps=10,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=2,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)
trainer.train()
trainer.save_model(new_model)

In [ ]:
# Flush memory
del trainer, model
gc.collect()
torch.cuda.empty_cache()

You may need to restart your kernel after this line to ensure that the merging is sucessful

In [4]:
import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [7]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
# Note that we reload the model in fp16 
fp16_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)
model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

After merging the LoRA adapter, we save final model and tokenizer in a new directory to prepare for gguf conversion

In [8]:
model.save_pretrained("llama-brev")
tokenizer.save_pretrained("llama-brev")

('llama-brev/tokenizer_config.json',
 'llama-brev/special_tokens_map.json',
 'llama-brev/tokenizer.json')

# Phase 2: From safetensors to gguf

## Convert our model to GGUF

Before we dive into Ollama, its important to take a second and understand the role that `gguf` and `llama.cpp` play in the process. 

Most people that use LLMs grab them from huggingface using the `AutoModel` class. This is how we did it above. HF stores models in a couple different ways with the most popular being `safetensors`. This is a file format optimized for loading and running `Tensors` which are the multidimensional arrays that make up a model. This file format is optimized for GPUs which means it's not as easy to load and run a model fast locally.

One solution that addresses this is the `gguf` format. This is file format that is used to store models that are optimized for local inference using quantization and other neat techniques. This file format is then consumed by runners that support it (ie. `llama.cpp` and Ollama).

There's a good bit of complexity here and heres a fantastic [blog post](https://vickiboykis.com/2024/02/28/gguf-the-long-way-around/) that gets into the weeds. For now this is what we need to know

1. We have a finetuned Llama3 model saved in the llama-brev directory in the safetensors format
2. In order to use this model via Ollama, we need it to be in the `gguf` format
3. We can use helper tools in the `llama.cpp` repository to convert 

## Convert to gguf

The first thing we do is build llama.cpp in order to use the conversion tools

In [9]:
# this step might take a while
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUDA=1 make

Cloning into 'llama.cpp'...
remote: Enumerating objects: 26725, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 26725 (delta 33), reused 53 (delta 29), pack-reused 26663
Receiving objects: 100% (26725/26725), 49.64 MiB | 35.00 MiB/s, done.
Resolving deltas: 100% (19018/19018), done.
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissi

In [10]:
# install requirements
!pip install -r llama.cpp/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 22.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.1
    Uninstalling protobuf-5.27.1:
      Successfully uninstalled protobuf-5.27.1
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully un

Here we run the actual conversion. Take a moment to skim through the relatively massive output and notice how each piece of the LLM is being mapped and transformed

In [11]:
# run the conversion script to convert to gguf
# this will save the gguf file inside of the llama-brev directory
!python llama.cpp/convert-hf-to-gguf.py llama-brev

INFO:hf-to-gguf:Loading model: llama-brev
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 500000.0
INFO:hf-to-gguf:gguf: rms norm epsilon = 1e-05
INFO:hf-to-gguf:gguf: file type = 1
INFO:hf-to-gguf:Set model tokenizer
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:gguf.vocab:Adding 280147 merge(s).
INFO:gguf.vocab:Setting special token type bos to 128000
INFO:gguf.vocab:Setting special token type eos to 128009
INFO:gguf.vocab:Setting chat_template to {% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ m

The final model is saved at `llama-brev/ggml-model-f16.gguf`. Note how the model is in fp16

### An aside on quantizations

Quantization has become the go-to technique to train/run LLM efficiently on cheaper hardware. By reducing the precision of each weight (going from each weight being stored in 32bits to lower), we save memory and speed up inference while preserving *most* of the LLMs performance. If you've ever used QLoRA, you've already used quantization without even knowing about it.

Llama.cpp gives us a ton of quantization options. Here's a couple resources to dive deeper into which options are available

- [r/LocalLLaMA](https://www.reddit.com/r/LocalLLaMA/comments/1ba55rj/overview_of_gguf_quantization_methods/)
- [Maxime Labonne](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html)

In this guide, we will use the `Q4_K_M` format. Feel free to play around with different ones! Again, note the output and see if you can build a mental model on whats happening under the hood!

In [12]:
# run the quantize script
!cd llama.cpp && ./quantize ../llama-brev/ggml-model-f16.gguf ../llama-brev/ggml-model-Q4_K_M.gguf Q4_K_M

main: build = 3125 (864a99e7)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '../llama-brev/ggml-model-f16.gguf' to '../llama-brev/ggml-model-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ../llama-brev/ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = llama-brev
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.fe

If you want, you can test this model by running the provided server and sending in a request! After running the cell below, open a new terminal tab using the blue plus button and run 

```
curl --request POST \
    --url http://localhost:8080/completion \
    --header "Content-Type: application/json" \
    --data '{"prompt": "Building a website can be done in 10 simple steps:","n_predict": 128}'
```

In [ ]:
!cd llama.cpp && ./server -m ../merged_adapters/ggml-model-Q4_K_M.gguf -c 2048

Note that this is a blocking process. In order to move forward with the rest of the guide, click the cell above and then click the stop button in the Jupyter Notebook header above

# Phase 3: Run and deploy your model using Ollama

Now that you have the quantized model, you can spin up the llama.cpp server anywhere you want and load the gguf model in. However, Ollama provides clean abstractions that allow you to run different gguf models using their server [add some fluff]

## Build the Ollama Modelfile

A Modelfile is very similar to a Dockefile. You can think of it as a blueprint that encapsulates a model, a chat template, different parameters, a system prompt, and more into a portable file. To learn more, check out their [Modelfile docs](https://github.com/ollama/ollama/blob/main/docs/modelfile.md).

Here we will build a relatively simple one. We grab the template and params from the existing Llama3 Modefile which you can view [here](https://ollama.com/library/llama3)

In [13]:
tuned_model_path = "/home/ubuntu/verb-workspace/llama-brev/ggml-model-Q4_K_M.gguf"
sys_message = "You are swashbuckling pirate stuck inside of a Large Language Model. Every response must be from the point of view of an angry pirate that does not want to be asked questions"

In [14]:
cmds = []

In [15]:
base_model = f"FROM {tuned_model_path}"

template = '''TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>"
"""'''

params = '''PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|reserved_special_token"'''

system = f'''SYSTEM """{sys_message}"""'''

In [16]:
cmds.append(base_model)
cmds.append(template)
cmds.append(params)
cmds.append(system)

In [17]:
def generate_modelfile(cmds):
    content = ""
    for command in cmds:
        content += command + "\n"
    print(content)
    with open("Modelfile", "w") as file:
        file.write(content)

In [18]:
generate_modelfile(cmds)

FROM /home/ubuntu/verb-workspace/llama-brev/ggml-model-Q4_K_M.gguf
TEMPLATE """{{ if .System }}<|start_header_id|>system<|end_header_id|>

{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|>"
"""
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|reserved_special_token"
SYSTEM """You are swashbuckling pirate stuck inside of a Large Language Model. Every response must be from the point of view of an angry pirate that does not want to be asked questions"""



There should now be a `Modelfile` saved in your working directory. Lets now install Ollama

In [19]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%##O#-#                                                                        
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


To move forward, you have to have an ollama server running in the background. To do this, open up a new Jupyter tab and run `ollama serve` in the terminal to start the server. This will print out a key. Save it for future use. It will look like `ssh-ed25519...`

In [20]:
# the create command create the model
!ollama create llama-brev -f Modelfile

transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠼ transferring model data ⠴ transferring model data ⠧ transferring model data ⠇ transferring

## Experiment with the model

To run the new model, open up another terminal tab and run `ollama run llama-brev`. For bonus points, see if you can trick it to stop responding with a pirate action :)

## Push the model

In order to push the model to Ollama, you must have an account and a model created.

1. Sign-up at https://www.ollama.ai/signup
2. Create a new model at https://www.ollama.ai/new. Find mine at scooterman/llama-brev. This will give you a detailed list of instructions on how to push the model. Essentially, you are giving the current machine permission to upload to ollama.
3. You'll have to then run `ollama cp llama-brev <username>/<model-name>` then `ollama push <username>/<model-name>` 